## Setup

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from geopy.distance import geodesic 

### Set library parameters

In [2]:
# Pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

### Import raw SF 311 data
Data downloaded from https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6 on February 6, 2020

In [3]:
df = pd.read_csv('../data/311_Cases.csv')
# df.head()

/Users/ihwang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## General statistics

According to https://www.dmv.ca.gov/portal/dmv/?1dmy&urile=wcm:path:/dmv_content_en/dmv/pubs/plates/standardplates, passenger vehicles can take on the following patterns (depending on year plate was issued):

NLLLNNN, NLLNNNN, LLNNNN (last two patterns for trailers only) where:
* N = any number between 0-9
* L = any letter between A-Za-z

We do a regular expression search for all of the above formats

*** Caveat: some license plates don't follow this format and many cases don't even list a license plate.

In [4]:
# Create an abandoned vehicles (AV) dataframe
df_av = df[(df['Category'] == 'Abandoned Vehicle') & (df['Request Details'].notna())]

Check the earliest case

In [5]:
pd.to_datetime(df_av['Opened']).min()

Timestamp('2008-07-01 02:07:11')

In [6]:
# Do a regular expression search for license plates
plates_modern = [re.search(r'[0-9]{1}[A-Za-z]{3}[0-9]{3}', str(row)) for row in df_av['Request Details']]
plates_perm_trailers = [re.search(r'[0-9]{1}[A-Za-z]{2}[0-9]{4}', str(row)) for row in df_av['Request Details']]
plates_trailers = [re.search(r'(^|\b)[A-Za-z]{2}[0-9]{4}', str(row)) for row in df_av['Request Details']]
plates_all = plates_modern + plates_perm_trailers + plates_trailers

# Convert all plates to uppercase
plates_all_upper = [plate.group(0).upper() for plate in plates_all if plate]

# Create a pandas series containing all uppercase license plates
se_plates = pd.Series(plates_all_upper)

# Print basic information
print('Total number of abandoned vehicles cases: ', len(df_av))
print('Number of abandoned vehicles cases that record license plate in known format: ', len(se_plates))
print('Number of unique license plates recorded in all abandoned vehicles cases: ', len(set(se_plates)))

Total number of abandoned vehicles cases:  233077
Number of abandoned vehicles cases that record license plate in known format:  95141
Number of unique license plates recorded in all abandoned vehicles cases:  60314


In [7]:
95141/233077

0.4081955748529456

It looks like just over 40% of abandoned vehicle cases recorded a license plate following one of the three types of DMV-listed plate formats. These cases contained license plate information for over 60,000 unique plates --- out of nearly 100,000 cases. This means that there must be some abandoned vehicles that were recorded in multiple cases.

Here's a glimpse at the top 10 most reported vehicles:

In [8]:
se_plates.value_counts().sort_values(ascending=False).head(10)

4AHL026    146
3LXY721    115
6PUJ756    87 
1RNW837    74 
6PUK177    65 
3XTE099    58 
6XVM291    58 
1PLM055    58 
6SLZ243    56 
7ZIB155    55 
dtype: int64

## Vehicles that have been called in repeatedly

In [9]:
for n_calls in [10, 20, 30, 50]:
    se_plates_counts = se_plates.value_counts()
    n_plates = len(se_plates_counts[se_plates_counts >= n_calls])
    print('Number of vehicles called in at least %d times: %d' %(n_calls, n_plates))

Number of vehicles called in at least 10 times: 548
Number of vehicles called in at least 20 times: 137
Number of vehicles called in at least 30 times: 46
Number of vehicles called in at least 50 times: 14


Let's take the vehicles that were called in at least 10 times and create a new dataframe out of them. While we're at it, let's make sure to have plate information handy (for color-coding, grouping, etc.) and to convert the date columns to datetime objects. For expedience, create one dataframe per plate, and then concatenate them all together in the end.

In [10]:
n = 10
plates_list = se_plates_counts[se_plates_counts >= n].index
list_df = []
for plate in plates_list:
    df_plate = df_av[df_av['Request Details'].str.contains(plate, case=False)].copy()
    df_plate['Opened'] = pd.to_datetime(df_plate['Opened'])
    df_plate['Closed'] = pd.to_datetime(df_plate['Closed'])
    df_plate['Updated'] = pd.to_datetime(df_plate['Updated'])
    df_plate.reset_index(inplace=True)
    df_plate['Plate'] = pd.Series([plate]*len(df_plate))
    df_plate.sort_values(by=['Opened'], ascending=False, inplace=True)
    list_df.append(df_plate)
df_repeats = pd.concat(list_df, ignore_index=True)
print('Between %d and %d, %d unique vehicles were called in as "abandoned" more than %s times each\n'
      %(
          df_repeats['Opened'].min().year,
          df_repeats['Opened'].max().year,
          len(plates_list),
          n
      )
     )
print('Vehicles called in at least %d times were categorized as:' %n)
print(*list(set(df_repeats['Request Type'])), sep='\n')

Between 2008 and 2020, 548 unique vehicles were called in as "abandoned" more than 10 times each

Vehicles called in at least 10 times were categorized as:
SSP Abandoned Vehicles
Abandoned Vehicle - Trailer
Abandoned Vehicle - Car2door
Abandoned Vehicle - DeliveryTruck
Abandoned Vehicle - Motorcycle
Abandoned Vehicle -
Abandoned Vehicle - SUV
Abandoned Vehicle - Car4door
Abandoned Vehicle - Other
Abandoned Vehicles High
Abandoned Vehicles
Abandoned Vehicle - PickupTruck


## Zooming in on trailers

We noticed that the vehicles included in df_repeats are often listed under different types of vehiles in the Request Type column. Our hypothesis is that people who are living in their vehicles are driving vehicles that are categorized as trailers, so let's narrow our focus to cars that were described at least once as a trailer.

In [12]:
df_types = df_repeats.groupby('Plate')['Request Type'].\
    value_counts(ascending=False).unstack()
trailers_plates = df_types[df_types['Abandoned Vehicle - Trailer'].notna()].index
df_trailers = df_repeats[df_repeats['Plate'].isin(trailers_plates)]

What kinds of vehicles are categorized as Trailers?

In [13]:
trailer_keywords = set([row.split('-')[1].strip().upper() + ' ' + row.split('-')[2].strip().upper() for row in df_trailers['Request Details']])
print('%s descriptions for vehicles listed at least once as "Abandoned Vehicle - Trailer":' %len(trailer_keywords))
print(*trailer_keywords, sep='\n')

1245 descriptions for vehicles listed at least once as "Abandoned Vehicle - Trailer":
ACURA MODIFIED BUS/TRUCK
JAMOOREE 
GMC CRUISE MASTER
RALLEYE N/A
BUS BUS
FORD PT CRUISER
TOYOTA ODESSY
HUMMER MILITARY
GMC 
MERCEDES CITATION
RV WINNEBAGO
CHEVY HORNET
ACURA MOTOR HOME
COMMANDER UNKN
ACURA LARGE RV
ITASCA FUN CRUISER
CHEVY NA
CHEVROLET DOLPHIN
DOLPHIN RV
CHEVROLET MOBILE HOME
PINNICLE MOTORHOME
UNKNOWN UKNOWN
UNKNOWN BORN FREE
CHEVROLET CAMPER VAN
WINNEBAGO WHITE
BLUE BIRD SCHOOL BUS
NISSAN 
CHEVROLET RV/CAMPER
WINNABEGO MINI WINI
TOYOTA MOTOR HOME
RV MOTOHOME
GMC WINNEBAGO
OLD RV RV
TOYOTA TRUCK CONNECTED TRAILER
ODYSEY MOTORHOME
GMC SEIRRA GMC SEIRRA
HUMMER 
ACURA DOLPHIN CAMPER
FORD 350
?? ??
TOYOTA DOLPHIN
SEA BREEZE NA
CHEVROLET CAVALIER YELLOWSTONE
ACURA FLAIR
UNKNOWN BOAT
PATHFINDER MOTOR HOME
SEABREEZE 
SPORTSMAN TIOGA
 BROUGHAM
VTOURING CROUSER
 PACE ARROW
SPRINTER MALARD MOTOR HOME
UNKNOWN UK
FORD UTILITY VAN
FORD RV RV
UNKNOWN VAN
CHEVROLET CAMPER/MOBILE HOME
TRAILOR OPEN F

In [14]:
print(
    'Of the %d vehicles called in as "abandoned" more than %s times each between %d and %d,'
    %(  
        len(plates_list),
        n,
        df_trailers['Opened'].min().year,
        df_trailers['Opened'].max().year, 
    )
)
print('%d of them were listed as `Abandoned - Trailer` at least once' %len(trailers_plates))

Of the 548 vehicles called in as "abandoned" more than 10 times each between 2008 and 2020,
156 of them were listed as `Abandoned - Trailer` at least once


In [15]:
156/548

0.2846715328467153

Let's examine how far these trailers traveled over time. First create dictionary `trailer_dict` containing information about total distance traveled and number of calls for each plate in `trailers_plates`.

In [16]:
trailer_dict = {}
for plate in trailers_plates:
    dist = 0
    df_trailer = df_trailers[(df_trailers['Plate'] == plate) & 
                             (df_trailers['Latitude'] != 0) & 
                             (df_trailers['Longitude'] != 0)].copy()
    df_trailer.sort_values('Opened', inplace=True)
    latlong_list = [tuple(row) for row in df_trailer[['Latitude', 'Longitude']].to_numpy()]
    for i in range(1,len(latlong_list)):
        dist += geodesic(latlong_list[i], latlong_list[i-1]).mi
    trailer_dict[plate] = {'total_dist': dist, 'num_calls': len(df_trailer)}

How far did these trailers travel, on average?

In [17]:
dist_list = [plate['total_dist'] for plate in trailer_dict.values()]
print('Average distance traveled by potential trailers: ', np.mean(dist_list))

Average distance traveled by potential trailers:  5.799780870279729


In [18]:
trailers_counts = df_trailers.groupby('Plate')['CaseID'].count().sort_values(ascending=False)
print('License plate %s was called in %d times between %s and %s'
      %(
          trailers_counts.index[0],
          trailers_counts[0],
          df_trailers[df_trailers['Plate'] == trailers_counts.index[0]].iloc[-1]['Opened'].strftime('%B %Y'),
          df_trailers[df_trailers['Plate'] == trailers_counts.index[0]].iloc[0]['Opened'].strftime('%B %Y')
      )
     )
print('It was most recently sighted in %s and described as a %s'
      %(
         
         df_trailers[df_trailers['Plate'] == trailers_counts.index[0]].iloc[0]['Opened'].strftime('%B %Y'),
         ' '.join(
             df_trailers[df_trailers['Plate'] == trailers_counts.index[0]].iloc[0]['Request Details'].split(' - ')[:3]
         )
      )
     )

License plate 4AHL026 was called in 146 times between May 2009 and August 2016
It was most recently sighted in August 2016 and described as a white Ford Chinook RV


In [19]:
most_recent = df_trailers['Opened'].max()
plate = df_trailers[df_trailers['Opened'] == most_recent].iloc[0]['Plate']
print('The most recently sighted trailer was plate %s' %plate)
print('It was sighted on %s and described as a %s' 
      %(
          most_recent.strftime('%B %d, %Y'),
          ' '.join(
             df_trailers[df_trailers['Opened'] == most_recent].iloc[0]['Request Details'].split(' - ')[:3]
          )
      )
     )
print('It had been called in %d times, starting in %s' 
      %(
          len(df_trailers[df_trailers['Plate'] == plate]),
          df_trailers[df_trailers['Plate'] == plate].iloc[-1]['Opened'].strftime('%B %d, %Y')
      )
     )
print('\nHere are the neighborhoods where it appeared: ')
print(df_trailers[df_trailers['Plate'] == plate]['Neighborhood'].value_counts(ascending=False))

The most recently sighted trailer was plate 7ZIB155
It was sighted on February 05, 2020 and described as a white Leprecaun? Leprecaun?
It had been called in 55 times, starting in August 16, 2017

Here are the neighborhoods where it appeared: 
Golden Gate Heights    51
Outer Sunset           3 
University Mound       1 
Name: Neighborhood, dtype: int64


Let's look at the calls for the plates that we found while reporting in San Francisco.

Eric Wicken's car cannot be found in our 311 call set

In [20]:
plate = '3GSK199'
plate in trailers_plates
plate in plates_list
df_av[df_av['Request Details'].str.contains(plate, case=False)]

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_jx4q_fizf,:@computed_region_bh8s_q3mv,:@computed_region_p5aj_wyqh,:@computed_region_fyvs_ahh9,:@computed_region_f58d_8dbm,:@computed_region_9dfj_4gjx,:@computed_region_vtsz_7cme,:@computed_region_n4xg_c4py,:@computed_region_sruu_94in,:@computed_region_4isq_27mq,:@computed_region_viu7_rrfi,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6ezc_tdp2,:@computed_region_6pnf_4xz7,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_y6ts_4iup,Neighborhoods


Randy Fata's car was not found in our list of trailers that have repeated calls, nor was it found in the list of vehicles that have been called in at least 20 times.

It was called in twice in September 2019 in Outer Richmond and Golden Gate Park. It was described as a `PickupTruck` and `Other`.

In [21]:
plate = 'D472817'
plate in trailers_plates
plate in plates_list
df_av[df_av['Request Details'].str.contains(plate, case=False)]

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_jx4q_fizf,:@computed_region_bh8s_q3mv,:@computed_region_p5aj_wyqh,:@computed_region_fyvs_ahh9,:@computed_region_f58d_8dbm,:@computed_region_9dfj_4gjx,:@computed_region_vtsz_7cme,:@computed_region_n4xg_c4py,:@computed_region_sruu_94in,:@computed_region_4isq_27mq,:@computed_region_viu7_rrfi,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6ezc_tdp2,:@computed_region_6pnf_4xz7,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_y6ts_4iup,Neighborhoods
3181250,11449687,09/22/2019 10:45:00 AM,09/25/2019 07:52:00 AM,09/25/2019 07:52:00 AM,Closed,DPT Abandoned Vehicles- Gone on Arrival - 09/24/2019 @ 1432 HRS # 85,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - PickupTruck,white - - SPRINTER - D472817,"7130 FULTON ST, SAN FRANCISCO, CA, 94121",FULTON ST,1.0,Outer Richmond,RICHMOND,37.771595,-122.508377,"(37.771595, -122.50837708)",Phone,NaN,8.0,8.0,4.0,29.0,2.0,11.0,9.0,55.0,6.0,26.0,26.0,NaN,8.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,8.0
4028931,11431552,09/18/2019 09:13:00 AM,09/20/2019 07:51:00 AM,09/20/2019 07:51:00 AM,Closed,DPT Abandoned Vehicles- Gone on Arrival - 09/19/2019 @ 1410 HRS # G85,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Other,White - Dodge - Sprinter - D472817,Intersection of FULTON ST and 47TH AVE,FULTON ST,1.0,Golden Gate Park,RICHMOND,37.771496,-122.507751,"(37.77149582, -122.50775146)",Phone,NaN,8.0,8.0,4.0,29.0,2.0,11.0,9.0,55.0,6.0,26.0,26.0,NaN,8.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,8.0


Eufemio's vehicle shows up in the dataframe of top called-in trailers.

In [22]:
plate = '2JPU300'
plate in trailers_plates
df_eufemio = df_trailers[df_trailers['Plate'] == plate]
print('Eufemio\'s vehicle has been called in %d times, starting in %d' 
      %(
          len(df_eufemio),
          df_eufemio['Opened'].min().year,
      )
     )
print('It was most recently called in on %s' %df_eufemio['Opened'].max().strftime('%B %d, %Y'))
print('\nHere are some of the more interesting Status Notes for a call on Eufemio\'s vehicle:')
print(df_eufemio.loc[948]['Status Notes'])
print(df_eufemio.loc[953]['Status Notes'])

Eufemio's vehicle has been called in 50 times, starting in 2016
It was most recently called in on November 01, 2019

Here are some of the more interesting Status Notes for a call on Eufemio's vehicle:
DPT Abandoned Vehicles - No Merit - 08-01-17 * DUE TO REEAT CALLS  , PCO IS MONITORING BLOCK   . VEH HAS NOT BEEN PARKED @ THIS LOCATION FOR 72 HRS
DPT Abandoned Vehicles - No Merit - * REPEAT CALL , PER CALLER , VEH  PARKED @ DIFFERENT LOCATIONS EVERY THREE DAYS . INDICATING MOVEMENT .  ALLOW THREE DAYS PARKED EFORE REPORTING


Wells' vehicle was not found in our list of trailers that have repeated calls, nor was it found in the list of vehicles that have been called in at least 20 times.

It was called in once in March 2017 in the Outer Sunset neighborhood. It was described as `Other`.

In [23]:
plate = '7P00592'
plate in trailers_plates
plate in plates_list
df_av[df_av['Request Details'].str.contains(plate, case=False)]

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_jx4q_fizf,:@computed_region_bh8s_q3mv,:@computed_region_p5aj_wyqh,:@computed_region_fyvs_ahh9,:@computed_region_f58d_8dbm,:@computed_region_9dfj_4gjx,:@computed_region_vtsz_7cme,:@computed_region_n4xg_c4py,:@computed_region_sruu_94in,:@computed_region_4isq_27mq,:@computed_region_viu7_rrfi,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6ezc_tdp2,:@computed_region_6pnf_4xz7,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_y6ts_4iup,Neighborhoods
1568829,6994367,03/31/2017 07:48:10 AM,04/04/2017 08:55:00 AM,04/04/2017 08:55:00 AM,Closed,DPT Abandoned Vehicles- Gone on Arrival - 04/03/2017 @ 1156 HRS BY #336,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Other,Silver - Ford - Van - 7P00592,"2194 48TH AVE, SAN FRANCISCO, CA, 94116",48TH AVE,4.0,Outer Sunset,TARAVAL,37.745566,-122.506883,"(37.7455663, -122.50688347)",Phone,NaN,39.0,10.0,7.0,35.0,3.0,1.0,5.0,29491.0,8.0,35.0,35.0,NaN,39.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0


This vehicle was not found in our list of trailers that have repeated calls, nor was it found in the list of vehicles that have been called in at least 20 times.

In [24]:
plate = '7ZUF129'
plate in trailers_plates
plate in plates_list

# Format dataframe a bit
df_unknown = df_av[df_av['Request Details'].str.contains(plate, case=False)].copy()
df_unknown['Opened'] = pd.to_datetime(df_unknown['Opened'])
df_unknown['Closed'] = pd.to_datetime(df_unknown['Closed'])
df_unknown['Updated'] = pd.to_datetime(df_unknown['Updated'])
df_unknown.reset_index(inplace=True)

print('This unknown vehicle has been called in %d times, starting in %d' 
      %(
          len(df_unknown),
          df_unknown['Opened'].min().year,
      )
     )
print('It was most recently called in on %s' %df_unknown['Opened'].max().strftime('%B %d, %Y'))
print('\nHere are some of the more interesting Status Notes for a call on this unknown vehicle:')
print(df_unknown.loc[13]['Status Notes'])
print(df_unknown.loc[11]['Status Notes'])

This unknown vehicle has been called in 16 times, starting in 2017
It was most recently called in on November 02, 2019

Here are some of the more interesting Status Notes for a call on this unknown vehicle:
DPT Abandoned Vehicles - Vehicle Complied w/Law - 10/09/2019   @ 1347 HRS # 120 * VEH NOW ON PACHECO
DPT Abandoned Vehicles - Vehicle Complied w/Law - VEH ALWAYS COMPLIES


In [25]:
df_unknown

,index,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_jx4q_fizf,:@computed_region_bh8s_q3mv,:@computed_region_p5aj_wyqh,:@computed_region_fyvs_ahh9,:@computed_region_f58d_8dbm,:@computed_region_9dfj_4gjx,:@computed_region_vtsz_7cme,:@computed_region_n4xg_c4py,:@computed_region_sruu_94in,:@computed_region_4isq_27mq,:@computed_region_viu7_rrfi,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6ezc_tdp2,:@computed_region_6pnf_4xz7,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,:@computed_region_y6ts_4iup,Neighborhoods
0,2130578,9747258,2018-10-31 13:57:00,2018-11-06 05:50:00,2018-11-06 05:50:00,Closed,DPT Abandoned Vehicles- Gone on Arrival - 11/05/2018 @ 1214 HRS # 130,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - PickupTruck,beige - Nissan - - 7ZUF129,"7633 GEARY BLVD, SAN FRANCISCO, CA, 94121",GEARY BLVD,1.0,Outer Richmond,RICHMOND,37.779125,-122.501396,"(37.77912521, -122.50139618)",Phone,NaN,8.0,8.0,4.0,29.0,2.0,11.0,9.0,55.0,6.0,26.0,26.0,NaN,8.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,8.0
1,2292260,9172858,2018-06-22 15:03:00,2018-07-03 07:20:00,2018-07-03 07:20:00,Closed,DPT Abandoned Vehicles- Gone on Arrival - 7-2-18 VEH GOA @ 1445 HOURS #320,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - PickupTruck,white - Nissan - odyssey - 7ZUF129,"1230 LA PLAYA, SAN FRANCISCO, CA, 94122",LA PLAYA,4.0,Outer Sunset,TARAVAL,37.763451,-122.509224,"(37.76345062, -122.50922394)",Phone,NaN,39.0,10.0,7.0,35.0,3.0,1.0,5.0,56.0,8.0,35.0,35.0,NaN,39.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0
2,2327444,8333996,2017-12-01 12:28:00,2017-12-07 17:25:00,2017-12-07 17:25:00,Closed,DPT Abandoned Vehicles- Gone on Arrival - VEH GOA 12/7/17 @1110 #320,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - PickupTruck,WHITE - - - 7ZUF129,"1270 LA PLAYA, SAN FRANCISCO, CA, 94122",LA PLAYA,4.0,Outer Sunset,TARAVAL,37.762703,-122.509171,"(37.76270294, -122.50917053)",Phone,NaN,39.0,10.0,7.0,35.0,3.0,1.0,5.0,56.0,8.0,35.0,35.0,NaN,39.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0
3,2341038,8287214,2017-11-19 10:31:00,2017-11-22 08:42:00,2017-11-22 08:42:00,Closed,DPT Abandoned Vehicles- Gone on Arrival - veh goa 11/21/17 @1115 #67,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Other,white - Winnebago - - 7ZUF129,"4710 IRVING ST, SAN FRANCISCO, CA, 94122",IRVING ST,4.0,Outer Sunset,TARAVAL,37.762299,-122.508568,"(37.76229858, -122.50856781)",Phone,NaN,39.0,10.0,7.0,35.0,3.0,1.0,5.0,56.0,8.0,35.0,35.0,NaN,39.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0
4,2770968,10886778,2019-05-20 14:01:00,2019-05-22 07:24:00,2019-05-22 07:24:00,Closed,DPT Abandoned Vehicles - Vehicle Complied w/Law - 05/21/2019 @ 1154 HRS # 62,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Other,White - Nissan - Unknown - 7ZUF129,"3714 RIVERA ST, SAN FRANCISCO, CA, 94116",RIVERA ST,4.0,Outer Sunset,TARAVAL,37.745564,-122.506332,"(37.74556351, -122.5063324)",Phone,NaN,39.0,10.0,7.0,35.0,3.0,1.0,5.0,29491.0,8.0,35.0,35.0,NaN,39.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,39.0
5,2797288,10818729,2019-05-05 07:36:00,2019-05-14 06:55:00,2019-05-14 06:55:00,Closed,DPT Abandoned Vehicles - Vehicle Complied w/Law - 05/13/2019 @ 1310 HRS # 120,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicle - Other,White - Nissan - Odyssey Camper - 7ZUF129,"2179 48TH AVE, SAN FRANCISCO, CA, 94116",48TH AVE,4.0,Outer Sunset,TARAVAL,37.745560,-122.507217,"(37.74555969, -122.50721741)",Phone,NaN,39.